In [1]:
#importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
# Importing dataset
df = pd.read_csv("energydata_complete.csv")
df.head

<bound method NDFrame.head of                  date  Appliances  lights         T1       RH_1         T2  \
0      11-01-16 17:00          60      30  19.890000  47.596667  19.200000   
1      11-01-16 17:10          60      30  19.890000  46.693333  19.200000   
2      11-01-16 17:20          50      30  19.890000  46.300000  19.200000   
3      11-01-16 17:30          50      40  19.890000  46.066667  19.200000   
4      11-01-16 17:40          60      40  19.890000  46.333333  19.200000   
...               ...         ...     ...        ...        ...        ...   
19730  27-05-16 17:20         100       0  25.566667  46.560000  25.890000   
19731  27-05-16 17:30          90       0  25.500000  46.500000  25.754000   
19732  27-05-16 17:40         270      10  25.500000  46.596667  25.628571   
19733  27-05-16 17:50         420      10  25.500000  46.990000  25.414000   
19734  27-05-16 18:00         430      10  25.500000  46.600000  25.264286   

            RH_2         T3      

In [3]:
df.drop(['date', 'lights'], 1, inplace = True)

In [4]:
#Firstly, we normalise our dataset to a common scale using the min max scaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
normalised_df = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
features_df = normalised_df.drop(columns=['Appliances'])
heating_target = normalised_df['Appliances']

In [5]:
#Now, we split our dataset into the training and testing dataset. Recall that we had earlier segmented the features and target variables.
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(features_df, heating_target, test_size=0.3, random_state=42)

In [6]:
#importing linear regression
from sklearn.linear_model import LinearRegression
linear_model = LinearRegression()

In [7]:
#fit the model to the training dataset
linear_model.fit(x_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [8]:
#obtain predictions
predicted_values = linear_model.predict(x_test)
predicted_values

array([0.03319482, 0.24412217, 0.0340092 , ..., 0.06843864, 0.09999407,
       0.05760656])

In [9]:
#MAE
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, predicted_values)
round(mae, 2)

0.05

In [10]:
#R-Squared
from sklearn.metrics import r2_score
r2_score = r2_score(y_test, predicted_values)
round(r2_score, 2)

0.15

In [11]:
#Residual Sum of Squares
import numpy as np
rss = np.sum(np.square(y_test - predicted_values))
round(rss, 2)

45.35

In [12]:
#Root Mean Square Error
from sklearn.metrics import  mean_squared_error
rmse = np.sqrt(mean_squared_error(y_test, predicted_values))
round(rmse, 3)

0.088

In [13]:
#comparing the effects of regularisation
def get_weights_df(model, feat, col_name):
  #this function returns the weight of every feature
  weights = pd.Series(model.coef_, feat.columns).sort_values()
  weights_df = pd.DataFrame(weights).reset_index()
  weights_df.columns = ['Features', col_name]
  weights_df[col_name].round(3)
  return weights_df

In [14]:
from sklearn.linear_model import Ridge
ridge_reg = Ridge(alpha = 0.5)
ridge_reg.fit(x_train, y_train)

Ridge(alpha=0.5, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

In [15]:
from sklearn.linear_model import Lasso
lasso_reg = Lasso(alpha = 0.001)
lasso_reg.fit(x_train, y_train)

Lasso(alpha=0.001, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [16]:
linear_model_weights = get_weights_df(linear_model, x_train, 'Linear_Model_Weight')
ridge_weights_df = get_weights_df(ridge_reg, x_train, 'Ridge_Weight')
lasso_weights_df = get_weights_df(lasso_reg, x_train, 'Lasso_weight')

final_weights = pd.merge(linear_model_weights, ridge_weights_df, on='Features')
final_weights = pd.merge(final_weights, lasso_weights_df, on='Features')
final_weights

,Features,Linear_Model_Weight,Ridge_Weight,Lasso_weight
0,RH_2,-0.456662,-0.401113,-0.000000
1,T_out,-0.321969,-0.250943,0.000000
2,T2,-0.236198,-0.193899,0.000000
3,T9,-0.189977,-0.188615,-0.000000
4,RH_8,-0.157585,-0.156584,-0.000110
5,RH_out,-0.077748,-0.050642,-0.049557
6,RH_7,-0.044620,-0.046297,-0.000000
7,RH_9,-0.039808,-0.041707,-0.000000
8,T5,-0.015645,-0.020714,-0.000000
9,T1,-0.003242,-0.021515,0.000000


In [17]:
ridge_reg = Ridge(alpha = 0.4)
ridge_reg.fit(x_train, y_train)
pred = ridge_reg.predict(x_test)

In [18]:
rmse = np.sqrt(mean_squared_error(y_test, pred))
round(rmse, 3)

0.088

In [19]:
lasso_reg = Lasso(alpha = 0.001)
lasso_reg.fit(x_train, y_train)
prediction = lasso_reg.predict(x_test)

In [20]:
lasso_weights_df = get_weights_df(lasso_reg, x_train, 'Lasso_weight')
nil = lasso_weights_df[lasso_weights_df['Lasso_weight'] != 0]
print(nil)

     Features  Lasso_weight
0      RH_out     -0.049557
1        RH_8     -0.000110
24  Windspeed      0.002912
25       RH_1      0.017880


In [21]:
rmse = np.sqrt(mean_squared_error(y_test, prediction))
round(rmse, 3)

0.094